<a href="https://colab.research.google.com/github/luisepifanio/my-useful-colab-notebooks/blob/features%2Foutlier-detection/outlier-detection/Outlier_Detection_using_FBProphet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Choose your execution environment
- If you want to use GPUs, please setup your environment before start
- First, you'll need to enable GPUs for the notebook:

1.    Navigate to Edit → Notebook Settings
2.    Select GPU from the Hardware Accelerator drop-down

<figure>
<center>
<img src='https://www.tutorialspoint.com/google_colab/images/enabling_gpu.jpg' />
<figcaption>Image Caption</figcaption></center>
</figure>

Next, we'll confirm that we can connect to the GPU with tensorflow:

 











In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()

# 2. Recreate folder  estructure about
Next step is about regenerate project structures to work:
- modules or relevant imports
- dataset to work

So, let's start downloading code and config

In [ ]:
from typing import List



base_url:str="https://raw.githubusercontent.com/luisepifanio/my-useful-colab-notebooks"
git_branch:str = "features/outlier-detection"
folder:str="outlier-detection"

#General folders
!mkdir -p "./modules"
!mkdir -p "./datasets"

# Files on root
environment_file:str = f"{base_url}/{git_branch}/{folder}/environment.yml"
files_on_root:List[str] = [environment_file]

for file in files_on_root:
  !wget --continue $file -P "." -nc

# code files on modules
files_on_modules:List[str] = [
  f"{base_url}/{git_branch}/{folder}/modules/outlier_utils.py"
]

for file in files_on_modules:
  !wget --continue $file -P "modules" -nc

#dataset download if needed
dsfiles:List[str] = [
  "https://drive.google.com/uc?export=download&id=1-R9ligkDV5Q0DkB3qh7VrFVqCv5SXRax"
]

for file in dsfiles:
  !wget --continue $file -P "datasets" -nc

#!ls -lhia sample_data

# 3. Mount google drive folder and use it if needed

Mount your google drive folder with data to use in your coding


In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

For instance locate your drive dataset and copy to your "local disk"


In [ ]:

#!gzip --keep /content/drive/MyDrive/datasets/conversion/conversion.sqlite
#!gunzip /content/drive/MyDrive/datasets/conversion/metrics.reindexed.csv.gz

!cp /content/drive/MyDrive/datasets/conversion/conversion.sqlite ./datasets/
# Check file got downloaded
!ls -lhia ./datasets

# 2. Install Conda on Google Colab

<!-- By Luis Epifanio luis.epifanio@ :) -->

`condacolab` simplifies the setup as much as possible, but there are some gotchas.

**⚠️ Read this before continuing!**

* The `condacolab` commands need to be run the first Code cell!
* Once you run `condacolab.install()`, the Python kernel will be restarted. This is **normal and expected**. After that, you can continue running the cells below like normal.
* Do not use the `Run all` option. Run the `condacolab` cell _individually_ and wait for the kernel to restart. **Only then**, you can run all cells if you want.
* You can only use the `base` environment. Do not try to create new ones; instead update `base` with either:
  * `conda install <packages>`
  * `conda env update -n base -f environment.yml`
* If you want to use GPUs, make sure you are using such an instance before starting!
* If you get an error, please raise an issue [here](https://github.com/jaimergp/condacolab/issues).

In [ ]:
!pip install -q condacolab
import condacolab
# Choose one of this, if you're in doubt choose first one
condacolab.install()
# condacolab.install_miniconda()
# condacolab.install_miniforge()
# condacolab.install_mambaforge()
# condacolab.install_anaconda()

In [ ]:
import condacolab
condacolab.check()

Optionally, you can refresh your environment dependenedcies using next command

In [ ]:
# !mamba install -q 
# TODO: Check how to update base environment from environment.yml
# On my case located at /content/drive/MyDrive/datasets/conversion/environment.yml
!mamba env update -n base -f environment.yml

In [ ]:
!conda clean -pt

Configure your imports to work

In [ ]:
# General imports
import sqlite3
import csv
from sqlite3 import Error
from typing import Dict, List, Any

import pandas as pd
from pandas import __version__ as pdversion
print(f'{"pandas": <14}: {pdversion: <16}' )

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import __version__ as pltversion
print(f'{"matplotlib": <14}: {pltversion: <16}' )
import seaborn as sns

from fbprophet import Prophet
from fbprophet import __version__ as fbversion
print(f'{"fbprophet": <14}: {fbversion: <16}' )
from datetime import datetime


In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()
# data_table.disable_dataframe_formatter() # or disable it

plt.rcParams["figure.figsize"] = [40, 10]
plt.style.use("ggplot")

Configure general vars 


## Load data

In [ ]:
def create_connection(db_file:str):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn


def execute_query(conn, query:str, params:Dict = None, verbose:bool = False) -> List[Dict[str, Any]]:
  try:
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()

    if params is not None:
      cur.execute( query, params )
    else :
      cur.execute( query )
    
    rows = cur.fetchall()
    result:List[Dict[str, Any]] = []

    for row in rows:
      d = dict(zip(row.keys(), row))   # a dict with column names as keys
      result.append(d)

    return result

  except Error as e:
    print(e)
    return None
  finally:
    if verbose:
      print(f"Statement executed:\n{query}\n")

In [ ]:
# General variables
wordkir:str = "datasets"
db_location:str = f"{wordkir}/conversion.sqlite"
conn = create_connection(db_location)

In [ ]:
query_tables:str = "SELECT name FROM sqlite_master;"

query:str = """
SELECT
  fecha,
  site,
  flow,
  device,
  Loading,
  Shipping,
  Payments,
  Review,
  Congrats,
  ROUND( ( Shipping * 1.0 / Loading  ) , 2 ) as LOAD_TO_SHP,
  ROUND( ( Payments * 1.0 / Shipping ) , 2 ) as SHP_TO_PAY,
  ROUND( ( Review   * 1.0 / Payments ) , 2 ) as PAY_TO_REV,
  ROUND( ( Congrats * 1.0 / Review   ) , 2 ) as REV_TO_CON,
  ROUND( ( Congrats * 1.0 / Shipping ) , 2 ) as TOTAL,
  CAST(strftime('%w', fecha) as integer) as day_of_week,
  CAST(strftime('%W', fecha) as integer) as week_of_year,
  CAST(strftime('%m', fecha) as integer) as month,
  CAST(strftime('%Y', fecha) as integer) as year,
  'Q' || COALESCE(NULLIF((SUBSTR(fecha, 4, 2) - 1) / 3, 0), 4) AS quarter
FROM traffic 
WHERE 
  fecha >= '2021-01-01'
  AND site IN ('MLM')
  AND flow IN ('cart')
  AND device IN ('android')
ORDER BY 
  fecha ASC,
  site,
  flow,
  device
"""

#
# Payments,
# Review,
# Congrats
#

# Load the data into a DataFrame
data = pd.read_sql_query(query, conn)
data["fecha"] = pd.to_datetime(data["fecha"])

data_table.DataTable(data, include_index=False, num_rows_per_page=24)

In [ ]:
# Adjust dataframe columns if needed
# df:pd.DataFrame = data[["fecha","Shipping"]]
# df.rename(columns={"fecha": "ds", "Shipping": "y"} , inplace = True)
# df.set_index("ds", inplace=True)
# df

In [ ]:
query_ds:str = """
SELECT
  strftime('%Y-%m-%dT%H:%M:%fZ', fecha) AS ds,
  Shipping AS y ,
  ROUND( ( Payments * 1.0 / Shipping ) , 2 ) as ratio
FROM traffic 
WHERE 
  fecha >= '2020-01-01'
  AND site IN ('MLM')
  AND flow IN ('cart')
  AND device IN ('android')
ORDER BY 
  fecha ASC,
  site,
  flow,
  device
"""
df = pd.read_sql_query(query_ds, conn)
df["ds"] = pd.to_datetime(df["ds"])

data_table.DataTable(df, include_index=False, num_rows_per_page=24)

In [ ]:
df.set_index("ds")["y"].plot();
plt.show()

In [ ]:
from modules.outlier_utils import prophet_fit, prophet_plot, get_outliers

In [ ]:
alpha=0.98

model = Prophet(
    interval_width=alpha, 
    yearly_seasonality=False, 
    weekly_seasonality=False, 
    #holidays=us_public_holidays, 
    changepoint_prior_scale=0.5
)

model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.add_seasonality(name='weekly', period=7, fourier_order=3, prior_scale=0.1)

today_index = 305
print('Cutoff date: ', df.index[today_index])

plt.rcParams["figure.figsize"] = [16, 9]
plt.style.use("ggplot")

predict_n = 14

fig, forecast, model = prophet_fit(df, model, today_index, predict_days=predict_n)

outliers, df_pred = get_outliers(df, forecast, today_index, predict_days=predict_n)

prophet_plot(df, fig, today_index, predict_days=predict_n, outliers=outliers)